In [94]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

print('Libraries imported.')

Libraries imported.


### Scraping code below with imports of beautiful soup

In [95]:
from bs4 import BeautifulSoup
#url itself for the wiki link
wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 

# fetch the content from the url, using the requests library
link_response=requests.get(wiki_link,timeout=5)    

# use the html parser to parse the url content and store it in a variable.
link_content=BeautifulSoup(link_response.content, "html.parser")

In [96]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
table=link_content.find_all('table')
df = pd.read_html(str(table))
neighborhoods=df[0]

In [97]:
neighborhoods=neighborhoods[neighborhoods.Borough != 'Not assigned']
neighborhoods.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [99]:
neighborhoods.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

### Joining Neighborhoods in the same Postcode

In [100]:
neighborhoods = neighborhoods.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()
neighborhoods.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [101]:
neighborhoods['Neighborhood'].loc[neighborhoods['Neighborhood'] == 'Not assigned'] = neighborhoods['Borough']

### Dimensions of the dataset are below

In [102]:
neighborhoods.shape

(103, 3)

In [103]:
neighborhoods.to_csv('Toronto.csv')